In [1]:
import pandas as pd
import geopandas as gpd
import rioxarray
import math
import shapely
import xvec

from shapely.geometry import Point, MultiPoint

In [2]:
sightlines = pd.read_pickle("my_results/method_1/d06_sightlines_geometries.pickle")
sightlines = sightlines[["start_point", "end_point", "sight_line_points"]]

# Convert tuples with coordinates to shapely geometry
Should eventually be deprecated by already using shapely geometries
in the first place.

In [3]:
def convert_to_point(row):
    return Point(row)

In [4]:
def convert_to_multipoint(row):
    if row:
        return MultiPoint(row)
    else:
        return None

In [5]:
# Convert start and end point to geometry
tuple_columns = ["start_point", "end_point"]

for col in tuple_columns:
    sightlines[col+"_geom"] = sightlines[col].apply(convert_to_point)


# Convert sight line points to geometry
sightlines["sl_points"] = sightlines["sight_line_points"].apply(convert_to_multipoint)

# Load raster dtm

In [6]:
raster = rioxarray.open_rasterio("data/dtm/dtm_raw/RGEALTI_FXX_1025_6285_MNT_LAMB93_IGN69.asc")
raster = raster.drop_vars("band").squeeze()

# Extract z coordinates from raster

In [7]:
# Extract z coords from raster
z_start = raster.drop_vars("spatial_ref").xvec.extract_points(points=sightlines["start_point_geom"], x_coords="x",y_coords="y")

# Extract z coords from raster
z_end = raster.drop_vars("spatial_ref").xvec.extract_points(points=sightlines["end_point_geom"], x_coords="x",y_coords="y")

In [ ]:
# TODO
# This is waayy to complicated. Should be reworked

new_gdf = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry')

for row in sightlines["sight_line_points_geom"]:
    if row is not None:
        points = row.geoms
        
        z_points = raster.drop_vars("spatial_ref").xvec.extract_points(points=points, x_coords="x",y_coords="y")
        z_points = z_points.xvec.to_geopandas().rename(columns={0: "z"})

        z_points["geometry"] = z_points.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
        z_points = z_points.drop(columns="z")
        
        multipoint = MultiPoint(z_points["geometry"].tolist())
        
        row_gdf = gpd.GeoDataFrame({'geometry': [multipoint]}, geometry='geometry')

        new_gdf = pd.concat([new_gdf, row_gdf], ignore_index=True)
    else:
        new_row = gpd.GeoDataFrame({'geometry': [None]}, geometry='geometry')
        new_gdf = pd.concat([new_gdf, new_row], ignore_index=True)